# Lab Assignment 3 — Graph Algorithms in Real-Life Applications

**Student:** Siddharth Kapoor

**Course:** ENCA351 — Design and Analysis of Algorithms Lab

This notebook implements four real-world graph problems (BFS/DFS friend suggestion, Bellman-Ford routing, Dijkstra emergency routing, and MST for cable installation), with profiling and visualizations. Dataset style: **Medium realistic** (~20 nodes).

Generated automatically.

In [ ]:
!pip install --quiet memory-profiler
import sys
import time
import random
import heapq
import tracemalloc
import math
from collections import deque, defaultdict
import matplotlib.pyplot as plt
print('Python', sys.version)
print('Libraries imported')

## Dataset: Medium realistic graphs

We'll build graphs of ~20 nodes for social network, city map, and office network. Random but reproducible weights/distances are used; edges are created to resemble real connectivity (sparse to moderately dense).

In [ ]:
random.seed(42)

# Create a medium-sized graph with 20 nodes
N = 20
nodes = [f"N{i}" for i in range(1, N+1)]

# Social network: undirected adjacency list
social_graph = {n: set() for n in nodes}
# create friendships with average degree ~4
for i in range(N):
    friends = random.sample(nodes, k=4)
    for f in friends:
        if f != nodes[i]:
            social_graph[nodes[i]].add(f)
            social_graph[f].add(nodes[i])

# Road network: directed weighted graph for Bellman-Ford (some negative edges)
road_edges = []
for i in range(N):
    out_deg = random.randint(2,4)
    dests = random.sample(nodes, k=out_deg)
    for d in dests:
        if d != nodes[i]:
            weight = random.randint(1, 20)
            # occasionally add a small negative weight to test Bellman-Ford
            if random.random() < 0.05:
                weight = -random.randint(1,5)
            road_edges.append((nodes[i], d, weight))

# Emergency map for Dijkstra: undirected positive-weight graph stored as adjacency list
emergency_graph = {n: [] for n in nodes}
for i in range(N):
    deg = random.randint(2,4)
    dests = random.sample(nodes, k=deg)
    for d in dests:
        if d != nodes[i]:
            t = random.randint(1,30)  # travel time
            emergency_graph[nodes[i]].append((d, t))
            emergency_graph[d].append((nodes[i], t))

# Cable installation: undirected weighted complete-ish graph (sparser)
cable_graph = []
for i in range(N):
    for j in range(i+1, N):
        if random.random() < 0.25:  # 25% edges to keep it sparse
            cost = random.randint(10, 100)
            cable_graph.append((nodes[i], nodes[j], cost))

print('Generated graphs: social_graph (adj sets), road_edges (list), emergency_graph (adj list), cable_graph (edge list)')

## Problem 1 — Social Network Friend Suggestion (BFS/DFS)

We will implement a friend-suggestion system using BFS to find 'friends of friends' who are not already connected.

In [ ]:
def suggest_friends_bfs(graph, user, max_depth=2):
    """
    Find friends-of-friends using BFS up to depth 2 (exclude existing friends and the user).
    Returns a dict mapping suggested node -> mutual friend count.
    """
    visited = set([user])
    q = deque([(user, 0)])
    suggestions = defaultdict(int)

    while q:
        node, depth = q.popleft()
        if depth >= max_depth:
            continue
        for nbr in graph[node]:
            if nbr == user:
                continue
            if depth + 1 == max_depth:
                # potential suggestion
                if nbr not in graph[user] and nbr != user:
                    suggestions[nbr] += 1
            if nbr not in visited:
                visited.add(nbr)
                q.append((nbr, depth+1))
    # sort by mutual count desc
    return dict(sorted(suggestions.items(), key=lambda x: -x[1]))

# Example
user = nodes[0]
print('User:', user)
print('Existing friends:', social_graph[user])
sugg = suggest_friends_bfs(social_graph, user)
print('\nSuggested friends (mutual_count):')
for k,v in list(sugg.items())[:10]:
    print(k, v)


## Problem 2 — Route Finding (Bellman-Ford)

We implement Bellman-Ford to find shortest paths from a source even with negative edges, and detect negative weight cycles.

In [ ]:
def bellman_ford(nodes, edges, src):
    # initialize
    dist = {n: float('inf') for n in nodes}
    dist[src] = 0
    predecessor = {n: None for n in nodes}

    for _ in range(len(nodes)-1):
        updated = False
        for u,v,w in edges:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                predecessor[v] = u
                updated = True
        if not updated:
            break

    # check for negative cycle
    neg_cycle = False
    for u,v,w in edges:
        if dist[u] + w < dist[v]:
            neg_cycle = True
            break

    return dist, predecessor, neg_cycle

# Run example
src = nodes[0]
dist, pred, neg = bellman_ford(nodes, road_edges, src)
print('Source:', src)
print('Negative cycle detected?' , neg)
print('Sample distances (first 10):')
for k in list(dist.keys())[:10]:
    print(k, dist[k])


## Problem 3 — Emergency Response (Dijkstra)

Dijkstra's algorithm for positive-weighted graphs using min-heap.

In [ ]:
def dijkstra(adj, src):
    dist = {n: float('inf') for n in adj}
    dist[src] = 0
    parent = {n: None for n in adj}
    heap = [(0, src)]
    while heap:
        d,u = heapq.heappop(heap)
        if d>dist[u]:
            continue
        for v,w in adj[u]:
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                parent[v] = u
                heapq.heappush(heap, (nd, v))
    return dist, parent

src = nodes[0]
dist_dij, parent_dij = dijkstra(emergency_graph, src)
print('Dijkstra distances (sample):')
for k in list(dist_dij.keys())[:10]:
    print(k, dist_dij[k])


## Problem 4 — Network Cable Installation (MST using Prim)

We implement Prim's algorithm with a priority queue to compute the minimum total cable cost.

In [ ]:
def prim_mst(nodes, edges):
    # build adjacency
    adj = {n: [] for n in nodes}
    for u,v,w in edges:
        adj[u].append((w,v))
        adj[v].append((w,u))
    start = nodes[0]
    visited = set([start])
    heap = []
    for w,v in adj[start]:
        heapq.heappush(heap, (w, start, v))
    total_cost = 0
    mst_edges = []
    while heap and len(visited) < len(nodes):
        w,u,v = heapq.heappop(heap)
        if v in visited:
            continue
        visited.add(v)
        total_cost += w
        mst_edges.append((u,v,w))
        for nw, nv in adj[v]:
            if nv not in visited:
                heapq.heappush(heap, (nw, v, nv))
    if len(visited) != len(nodes):
        return None, None  # graph not connected
    return total_cost, mst_edges

cost, selected = prim_mst(nodes, cable_graph)
print('MST total cost:', cost)
print('Sample MST edges (first 10):', selected[:10])


## Profiling & Visualization

We measure execution time and memory usage (using `tracemalloc` as a fallback) for each algorithm.

In [ ]:
def profile_func(func, *args, **kwargs):
    tracemalloc.start()
    t0 = time.perf_counter()
    result = func(*args, **kwargs)
    t1 = time.perf_counter()
    mem_current, mem_peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return {'time_s': t1-t0, 'mem_current_bytes': mem_current, 'mem_peak_bytes': mem_peak, 'result': result}

# Profile each algorithm on the medium graph
profiles = {}
profiles['friend_suggestion'] = profile_func(suggest_friends_bfs, social_graph, nodes[0])
profiles['bellman_ford'] = profile_func(bellman_ford, nodes, road_edges, nodes[0])
profiles['dijkstra'] = profile_func(dijkstra, emergency_graph, nodes[0])
profiles['prim_mst'] = profile_func(prim_mst, nodes, cable_graph)

for k,v in profiles.items():
    print(k, '- time(s):', round(v['time_s'],6), 'peak_mem(bytes):', v['mem_peak_bytes'])


### Execution time vs Graph Size

We run a small experiment varying number of nodes (10, 50, 100) to visualize scaling behavior. For demonstration we will generate random graphs scaled up and measure runtime.

In [ ]:
def make_random_graph(n, avg_deg=3):
    nodes = [f'N{i}' for i in range(n)]
    # adjacency for emergency (undirected)
    eg = {x: [] for x in nodes}
    for i in range(n):
        deg = min(n-1, avg_deg + random.randint(0,2))
        dests = random.sample(nodes, k=deg)
        for d in dests:
            if d!=nodes[i]:
                t = random.randint(1,20)
                eg[nodes[i]].append((d,t))
                eg[d].append((nodes[i], t))
    return nodes, eg

sizes = [10, 50, 100]
results_time = {'dijkstra': [], 'prim': []}
for s in sizes:
    ns, eg = make_random_graph(s, avg_deg=3)
    # convert eg to a smaller edge list for prim
    edge_set = set()
    for u in eg:
        for v,w in eg[u]:
            a,b = sorted([u,v])
            edge_set.add((a,b,w))
    edges_list = list(edge_set)

    pd = profile_func(dijkstra, eg, ns[0])
    pp = profile_func(prim_mst, ns, edges_list)
    results_time['dijkstra'].append(pd['time_s'])
    results_time['prim'].append(pp['time_s'])

print('Sizes:', sizes)
print('Dijkstra times:', results_time['dijkstra'])
print('Prim times:', results_time['prim'])

plt.figure(figsize=(8,4))
plt.plot(sizes, results_time['dijkstra'], marker='o')
plt.title('Dijkstra: Execution time vs |V|')
plt.xlabel('Number of nodes')
plt.ylabel('Time (s)')
plt.grid(True)
plt.show()

plt.figure(figsize=(8,4))
plt.plot(sizes, results_time['prim'], marker='o')
plt.title('Prim (MST): Execution time vs |V|')
plt.xlabel('Number of nodes')
plt.ylabel('Time (s)')
plt.grid(True)
plt.show()


## Final Notes & Next Steps

This notebook provides the required algorithm implementations, profiling, and plots for a medium-sized dataset. For submission, you may:

1. Add more documentation and discussion cells (complexity analysis, reflections).
2. Push the notebook, README.md and requirements.txt to your GitHub repository.

The files `graph_realworld.ipynb`, `README.md`, and `requirements.txt` are saved to `/mnt/data/`.